In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
train = pd.read_csv('../input/train_set.csv')
test = pd.read_csv('../input/test_set.csv')

In [4]:
tok = lambda x: x.split()
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tok,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train['word_seg'])
test_term_doc = vec.transform(test['word_seg'])

In [5]:
trn_term_doc, test_term_doc

(<102277x2820705 sparse matrix of type '<class 'numpy.float64'>'
 	with 75740957 stored elements in Compressed Sparse Row format>,
 <102277x2820705 sparse matrix of type '<class 'numpy.float64'>'
 	with 73583074 stored elements in Compressed Sparse Row format>)

In [6]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [7]:
x = trn_term_doc
test_x = test_term_doc

In [11]:
train.head()

,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


In [12]:
y_value = train['class'].values

In [15]:
def get_mdl(v):
    y = (y_value == v).astype(int)
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True, n_jobs=4)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [16]:
preds = np.zeros((len(test), 19))

In [17]:
for i in range(19):
    j = i + 1
    print('fit', j)
    m,r = get_mdl(j)
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit 1


/home/xiaqiang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(self.n_jobs))


fit 2
fit 3
fit 4
fit 5
fit 6
fit 7
fit 8
fit 9
fit 10
fit 11
fit 12
fit 13
fit 14
fit 15
fit 16
fit 17
fit 18
fit 19


In [18]:
test['class'] = np.argmax(preds,1)

In [19]:
test.to_csv('submission_nb_svm.csv',columns=['id', 'class'], index=False)